### Imports

In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
import transformers

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

2023-11-29 16:45:03.574276: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-29 16:45:03.631987: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load and test out mistral-7b

In [2]:
model_name='mistralai/Mistral-7B-Instruct-v0.1'

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [3]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [4]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [6]:
!pip list

Package                                  Version
---------------------------------------- ---------------
absl-py                                  1.4.0
accelerate                               0.21.0
aiohttp                                  3.8.5
aiohttp-cors                             0.7.0
aiorwlock                                1.3.0
aiosignal                                1.3.1
ansiwrap                                 0.8.4
anyio                                    3.7.1
apache-beam                              2.46.0
argon2-cffi                              21.3.0
argon2-cffi-bindings                     21.2.0
array-record                             0.4.1
arrow                                    1.2.3
asttokens                                2.2.1
astunparse                               1.6.3
async-lru                                2.0.4
async-timeout                            4.0.3
attrs                                    23.1.0
Babel                                    2.

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [6]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


In [7]:
device = 'cuda'

messages = [
    {"role": "user", "content": "What can you tell me about the 2021 fantasy football season?"},
    {"role": "assistant", "content": """The 2021 fantasy football season began on August 17th, 2021, with the first games of the NFL Preseason. The regular season started on September 11th, 2021 and ended on January 9th, 2022. The Super Bowl LVI, which was the 56th Super Bowl, was held on February 6th, 2022, at SoFi Stadium in Los Angeles, California.

    Due to the COVID-19 pandemic, the 2021 fantasy football season was played with some changes, including a reduced preseason schedule, the elimination of in-person attendance at games, and the use of virtual meetings for teams.

    In terms of fantasy football leagues, many leagues opted to use a PPR (Point Pertinent) format, which rewards players for receiving passing yards in addition to rushing yards. The use of PPR has become increasingly popular in recent years, as it provides more opportunities for players to score points.

    As for notable players in the 2021 fantasy football season, some of the top performers included:

    * Quarterbacks: Patrick Mahomes of the Kansas City Chiefs, Justin Herbert of the Los Angeles Chargers, and Tom Brady of the Tampa Bay Buccaneers.
    * Running Backs: Christian McCaffery of the Carolina Panthers, Alvin Kamara of the New Orleans Saints, and Dalvin Cook of the Minnesota Vikings.
    * Wide Receivers: Tyreek Hill of the Kansas City Chiefs, Stefon Diggs of the Buffalo Bills, and DeAndre Hopkins of the Arizona Cardinals.
    * Tight Ends: Travis Kelce of the Kansas City Chiefs, George Kittle of the San Francisco 49ers, and Tyler Higbee of the Los Angeles Rams.

    In terms of team performance, the Tampa Bay Buccaneers won the Super Bowl LVI, defeating the Kansas City Chiefs 31-9. The Green Bay Packers finished the regular season with the best record (13-3), but were eliminated in the divisional round of the playoffs by the Los Angeles Rams. The Oakland Raiders, who moved to Las Vegas in 2020, had a strong season, finishing with a record of 10-6 and making it to the playoffs for the first time since 2016."""},
    {"role": "user", "content": "How many points did Maholmes score that season?"},
    {"role": "assistant", "content": "Patrick Mahomes, the quarterback for the Kansas City Chiefs, scored a total of 456 points in the regular season of the 2021 fantasy football season. This made him the top scoring quarterback in the PPR (Point Pertinent) format, which rewards players for receiving passing yards in addition to rushing yards."},
    {"role": "user", "content": "Can you tell me how many points he scored in week 3 of that season?"},
    {"role": "assistant", "content": "Patrick Mahomes scored 35 points in week 3 of the 2021 fantasy football season. This was a strong performance, as Mahomes completed 18 of 29 passes for 283 yards and three touchdowns in a win over the Indianapolis Colts."},
    {"role": "user", "content": "Did anyone score more than him in that week?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What can you tell me about the 2021 fantasy football season? [/INST]The 2021 fantasy football season began on August 17th, 2021, with the first games of the NFL Preseason. The regular season started on September 11th, 2021 and ended on January 9th, 2022. The Super Bowl LVI, which was the 56th Super Bowl, was held on February 6th, 2022, at SoFi Stadium in Los Angeles, California.

    Due to the COVID-19 pandemic, the 2021 fantasy football season was played with some changes, including a reduced preseason schedule, the elimination of in-person attendance at games, and the use of virtual meetings for teams.

    In terms of fantasy football leagues, many leagues opted to use a PPR (Point Pertinent) format, which rewards players for receiving passing yards in addition to rushing yards. The use of PPR has become increasingly popular in recent years, as it provides more opportunities for players to score points.

    As for notable players in the 2021 fantasy football season, som

In [8]:
inputs_not_chat = tokenizer.encode_plus("[INST] Tell me about fantasy football? [/INST]", return_tensors="pt")['input_ids'].to(device)

generated_ids = model.generate(inputs_not_chat, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Tell me about fantasy football? [/INST] Fantasy football is a sport in which players create teams of imaginary players and play against each other in a competition. The players are chosen from real-life professional football teams and are given various attributes such as points scored, yards gained, and receptions. Each player has a set number of points that they can earn based on their performance in games.

The players on a fantasy football team are chosen based on their perceived potential for success in the real world. For example, if a player is known to be a powerful running back or a skilled receiver, they would be a valuable addition to a fantasy football team.

The game is typically played over the course of a season, with each week's games determining the points earned by each player on each team. At the end of the season, the team with the most points wins the championship.

Fantasy football has become increasingly popular in recent years, with millions of people 

### RAG RAG RAG

In [9]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import FAISS

Example ChatPromptTemplate

```python
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)
```

In [33]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=300,
)


In [34]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)



In [35]:
prompt_template = """
### [INST] Instruction: Answer the question based on your fantasy football knowledge. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
 """

In [36]:
# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [37]:
from langchain.document_loaders import AsyncChromiumLoader, UnstructuredURLLoader, PlaywrightURLLoader
from langchain.document_loaders.blob_loaders.schema import Blob
from langchain.document_loaders.parsers.txt import TextParser
from langchain.document_loaders.url_selenium import SeleniumURLLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.text_splitter import CharacterTextSplitter

In [85]:
test_document = """
ALLEN PARK, Mich. -- Detroit Lions running back Jahmyr Gibbs has been ruled out for Sunday's game against the Tampa Bay Buccaneers with a hamstring injury while Pro Bowl receiver Amon-Ra St. Brown (abdomen) is set to make his return.

Gibbs, who did not practice this week, also missed last week's 42-24 victory over the Carolina Panthers with the same injury. With 179 yards and a touchdown with an additional 70 receiving yards in his four appearances, he is Detroit's second-leading rusher.

Veteran running back David Montgomery has carried the load in his absence, with back-to-back games of at least 100 yards rushing. Detroit has rushed for nine touchdowns through the first five games of the season, tying a team record.

Although expectations are high for Gibbs as the Lions' No. 12 overall selection, running back coach Scottie Montgomery is helping him develop at his own pace.

"He's right where I thought he would be, he's right where he should be coming along at at this time," Scottie Montgomery said.

"We don't judge players based off of anything else besides achievement. Like there is success and there is expectations and I want people outside the building to put those expectations on him because, guess what, whether we believe it or not it motivates him. It does and that's a really, really good tool."

Meanwhile, rookie safety Brian Branch, who has an ankle injury, is also set to miss his second consecutive game. The second-round pick out of Alabama was held out of practice for the entire week. With 25 tackles, a pick-six and 4 passes defended, he is Detroit's fourth-leading tackler.

Rookie tight end Sam LaPorta is questionable with a calf strain that flared up Wednesday, forcing him to miss Thursday's practice. He said it likely stemmed from Sunday's win over Carolina, and he intends to play Sunday.

"It's football, things happen, we certainly work our bodies pretty hard so I'm feeling a lot better today," LaPorta said.
"""

In [86]:
raw_documents = TextParser().parse(Blob(data=test_document))

In [87]:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, HuggingFaceEmbeddings())

Created a chunk of size 234, which is longer than the specified 100
Created a chunk of size 258, which is longer than the specified 100
Created a chunk of size 233, which is longer than the specified 100
Created a chunk of size 158, which is longer than the specified 100
Created a chunk of size 126, which is longer than the specified 100
Created a chunk of size 307, which is longer than the specified 100
Created a chunk of size 286, which is longer than the specified 100
Created a chunk of size 220, which is longer than the specified 100


```python
HuggingFaceEmbeddings(
    *,
    client: Any = None,
    model_name: str = 'sentence-transformers/all-mpnet-base-v2',
    cache_folder: Optional[str] = None,
    model_kwargs: Dict[str, Any] = None,
    encode_kwargs: Dict[str, Any] = None,
    multi_process: bool = False,
) -> None
```

In [88]:
retriever = db.as_retriever()

In [93]:
retriever.invoke("who is gibbs")

[Document(page_content="ALLEN PARK, Mich. -- Detroit Lions running back Jahmyr Gibbs has been ruled out for Sunday's game against the Tampa Bay Buccaneers with a hamstring injury while Pro Bowl receiver Amon-Ra St. Brown (abdomen) is set to make his return.", metadata={'source': None}),
 Document(page_content="Although expectations are high for Gibbs as the Lions' No. 12 overall selection, running back coach Scottie Montgomery is helping him develop at his own pace.", metadata={'source': None}),
 Document(page_content="Gibbs, who did not practice this week, also missed last week's 42-24 victory over the Carolina Panthers with the same injury. With 179 yards and a touchdown with an additional 70 receiving yards in his four appearances, he is Detroit's second-leading rusher.", metadata={'source': None}),
 Document(page_content='"He\'s right where I thought he would be, he\'s right where he should be coming along at at this time," Scottie Montgomery said.', metadata={'source': None})]

In [38]:
chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [92]:
query = "What did Laporta say?"
docs = db.similarity_search(query)
print(docs[0].page_content)

"It's football, things happen, we certainly work our bodies pretty hard so I'm feeling a lot better today," LaPorta said.


In [21]:
llm_chain.invoke({"context":"", "question": "Did Gibbs start this week?"})

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': '',
 'question': 'Did Gibbs start this week?',
 'text': " I'm sorry, but I don't have access to real-time information about specific players in fantasy football. Can you please provide me with more context or a specific date and time frame so that I can give you an accurate answer?"}

In [91]:
chain.invoke("What did Laporta say?")

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': [Document(page_content='"It\'s football, things happen, we certainly work our bodies pretty hard so I\'m feeling a lot better today," LaPorta said.', metadata={'source': None}),
  Document(page_content="Rookie tight end Sam LaPorta is questionable with a calf strain that flared up Wednesday, forcing him to miss Thursday's practice. He said it likely stemmed from Sunday's win over Carolina, and he intends to play Sunday.", metadata={'source': None}),
  Document(page_content='"We don\'t judge players based off of anything else besides achievement. Like there is success and there is expectations and I want people outside the building to put those expectations on him because, guess what, whether we believe it or not it motivates him. It does and that\'s a really, really good tool."', metadata={'source': None}),
  Document(page_content='"He\'s right where I thought he would be, he\'s right where he should be coming along at at this time," Scottie Montgomery said.', metadata={'so

#### Let's try to load the same article with LangChains web crawler loaders

In [15]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
articles = ["https://www.fantasypros.com/2023/11/rival-fantasy-nfl-week-10/",
            "https://www.fantasypros.com/2023/11/5-stats-to-know-before-setting-your-fantasy-lineup-week-10/",
            "https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/",
            "https://www.fantasypros.com/2023/11/nfl-dfs-week-10-stacking-advice-picks-2023-fantasy-football/",
             "https://www.fantasypros.com/2023/11/players-to-buy-low-sell-high-trade-advice-2023-fantasy-football/"]

In [28]:
loader = AsyncChromiumLoader(articles)
docs = loader.load()

In [29]:
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

In [30]:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
documents = text_splitter.split_documents(docs_transformed)
db = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

Created a chunk of size 146, which is longer than the specified 100
Created a chunk of size 4117, which is longer than the specified 100
Created a chunk of size 567, which is longer than the specified 100
Created a chunk of size 316, which is longer than the specified 100
Created a chunk of size 136, which is longer than the specified 100
Created a chunk of size 249, which is longer than the specified 100
Created a chunk of size 113, which is longer than the specified 100
Created a chunk of size 449, which is longer than the specified 100
Created a chunk of size 137, which is longer than the specified 100
Created a chunk of size 201, which is longer than the specified 100
Created a chunk of size 132, which is longer than the specified 100
Created a chunk of size 479, which is longer than the specified 100
Created a chunk of size 216, which is longer than the specified 100
Created a chunk of size 138, which is longer than the specified 100
Created a chunk of size 172, which is longer th

What do these documents look like?


In [48]:
documents[:5]

[Document(page_content='Skip to main content  Skip to navigation', metadata={'source': 'https://www.espn.com/nfl/story/_/id/38652877/lions-rb-jahmyr-gibbs-wr-amon-ra-st-brown-return-vs-bucs'}),
 Document(page_content="Top Events  NCAAF  MLB  NFL  WNBA  F1  NHL  NBA  PGA Tour  NASCAR  Tennis (W)\nLPGA Tour  NCAAM  Tennis (M)  UFC  NCAAW  Top Soccer  PLL (Lacrosse)  ICC\nCricket World Cup  Men's NCAA Ice Hockey", metadata={'source': 'https://www.espn.com/nfl/story/_/id/38652877/lions-rb-jahmyr-gibbs-wr-amon-ra-st-brown-return-vs-bucs'}),
 Document(page_content='NFL', metadata={'source': 'https://www.espn.com/nfl/story/_/id/38652877/lions-rb-jahmyr-gibbs-wr-amon-ra-st-brown-return-vs-bucs'}),
 Document(page_content="* Top Events \n  * NCAAF \n  * MLB \n  * NFL \n  * WNBA \n  * F1 \n  * NHL \n  * NBA \n  * PGA Tour \n  * NASCAR \n  * Tennis (W) \n  * LPGA Tour \n  * NCAAM \n  * Tennis (M) \n  * UFC \n  * NCAAW \n  * Top Soccer \n  * PLL (Lacrosse) \n  * ICC Cricket World Cup \n  * Men's NC

In [31]:
retriever = db.as_retriever()
retriever.invoke("what do you think about alvin kamara?")


[Document(page_content='Averaging 45.3 receiving yards per game this season, Alvin Kamara has been a\nfocal point of the aerial attack with Derek Carr', metadata={'source': 'https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/'}),
 Document(page_content='at quarterback. Now armed with an upgrade under Center and multiple weapons at\nwideout to keep the defensive attention off him, Kamara is back to elite\nreceiving numbers for a running back. Kamara has recorded 36 or more receiving\nyards in four of his last five games and has topped 33 or more in five of six\ngames this season. He is being peppered with targets regardless of opponent\nand should be leaned on again in Week 10. Tap the More on Kamara for Week 10.', metadata={'source': 'https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/'}),
 Document(page_content='## **Sleeper Picks NFL Week 10**\n\n_Season record: 16-20_\n\n### **Alvin Kamara (RB – NO)', metadata={'s

Super noisy but let's still see how it performs?

In [26]:
llm_chain.invoke({"context":"", "question": "Should I pick up Alvin Kamara for my fantasy team?"})

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': '',
 'question': 'Should I pick up Alvin Kamara for my fantasy team?',
 'text': '\nWhether or not you should pick up Alvin Kamara for your fantasy team depends on a few factors, such as the specific league rules and roster requirements, the current performance of Kamara and other players in your league, and your overall strategy for building your team.\n\nIn general, Kamara is considered to be one of the top running backs in the NFL and has had a strong season so far. He has been a consistent producer of points for his owners, with multiple touchdown'}

In [39]:
retriever = db.as_retriever()

chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [27]:
chain.invoke("Should I pick up Alvin Kamara for my fantasy team?")

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': [Document(page_content='at quarterback. Now armed with an upgrade under Center and multiple weapons at\nwideout to keep the defensive attention off him, Kamara is back to elite\nreceiving numbers for a running back. Kamara has recorded 36 or more receiving\nyards in four of his last five games and has topped 33 or more in five of six\ngames this season. He is being peppered with targets regardless of opponent\nand should be leaned on again in Week 10. Tap the More on Kamara for Week 10.', metadata={'source': 'https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/'}),
  Document(page_content='## **Sleeper Picks NFL Week 10**\n\n_Season record: 16-20_\n\n### **Alvin Kamara (RB – NO)', metadata={'source': 'https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/'}),
  Document(page_content=', as he should have plenty of field goal opportunities to both miss and make.\nAt running back, I will be going with CMC and Al

In [40]:
chain.invoke("I have Josh Jacobs, should I trade him for Kareem Hunt?")

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': [Document(page_content='“ **Josh Jacobs** is coming off his second-best game of the season in week\nnine. While Jacobs has had more success, he still runs at 3.2 YPC. I can’t\nimagine the team competing with a rookie quarterback. Jocabs has the sixth-\nhardest remaining schedule of the season for fantasy running backs. Let’s not\nforget he still has a bye week to deal with. I’d be looking to sell high on\nJacobs to acquire like De’Von Achane or Tony Pollard', metadata={'source': 'https://www.fantasypros.com/2023/11/players-to-buy-low-sell-high-trade-advice-2023-fantasy-football/'}),
  Document(page_content='Kareem Hunt is the fourth highest when it relates to inside the 5 rushing\nattempt percentage in the ENTIRE NFL. He has played six games and didn’t even\nstart on a roster to begin the 2023 season. With the Cleveland Browns, Jerome\nFord has been pushed into the “Between the 20s” running back who needs a\nhomerun shot to bring your fantasy team a ceiling play. Kareem Hun

In [42]:
chain.invoke("What are your thoughts on Ja'Marr Chase going into this week?")

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': [Document(page_content='### **Ja’Marr Chase (WR – CIN)\n\n**', metadata={'source': 'https://www.fantasypros.com/2023/11/players-to-buy-low-sell-high-trade-advice-2023-fantasy-football/'}),
  Document(page_content="Ja'Marr Chase\n\n(back) at practice Friday\n\nAmon-Ra St. Brown", metadata={'source': 'https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/'}),
  Document(page_content="Ja'Marr Chase\n\n(back) at practice Friday\n\nAmon-Ra St. Brown", metadata={'source': 'https://www.fantasypros.com/2023/11/5-stats-to-know-before-setting-your-fantasy-lineup-week-10/'}),
  Document(page_content="Ja'Marr Chase\n\n(back) at practice Friday\n\nAmon-Ra St. Brown", metadata={'source': 'https://www.fantasypros.com/2023/11/nfl-dfs-week-10-stacking-advice-picks-2023-fantasy-football/'})],
 'question': "What are your thoughts on Ja'Marr Chase going into this week?",
 'text': " Based on the information provided, it seems that Ja'Marr Chase is currently pract

Sweet, still gives the right answer! This may cause issues as we add more docs

### Conversational Retrieval Chain

In [30]:
from langchain.schema.runnable import RunnableMap
from langchain.schema import format_document

In [31]:
from langchain.prompts.prompt import PromptTemplate

_template = """[INST] Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:
[/INST]"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [32]:
template = """[INST]Answer the question based only on the following context and chat history:
Context:
{context}

Chat History:
{chat_history}

Question: {question}
[/INST]
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [33]:
# Create llm chain 
llm_chain_cq = LLMChain(llm=mistral_llm_0_temp, prompt=CONDENSE_QUESTION_PROMPT)

# Create llm chain 
llm_chain_a = LLMChain(llm=mistral_llm, prompt=ANSWER_PROMPT)

In [34]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")
def _combine_documents(docs, document_prompt = DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [35]:
from typing import Tuple, List
def _format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "Me: " + dialogue_turn[0]
        ai = "Assistant: " + dialogue_turn[1]
        buffer += "\n" + "\n".join([human, ai])
    return buffer

In [36]:
RunnableMap(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: _format_chat_history(x['chat_history']))
    )   

{
  standalone_question: RunnableAssign(mapper={
                         chat_history: RunnableLambda(...)
                       })
}

In [37]:
def get_text_from_standalone_question(data):
    return data['standalone_question']['text']

def get_chathistory_from_standalone_question(data):
    return data['standalone_question']['chat_history']

_inputs = RunnableMap(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: _format_chat_history(x['chat_history'])
    ) | llm_chain_cq ,
)
_context = {
    "chat_history": get_chathistory_from_standalone_question,
    "context": get_text_from_standalone_question  | retriever | _combine_documents,
    "question": get_text_from_standalone_question
}
conversational_qa_chain = _inputs | _context | llm_chain_a

In [38]:
_inputs.invoke({
    "question": "What was Gibbs injury?",
    "chat_history": [],
})

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'standalone_question': {'question': 'What was Gibbs injury?',
  'chat_history': '',
  'text': " What was Gibbs' injury?"}}

In [49]:
conversational_qa_chain.invoke({
    "question": "What does active roster mean in NFL?",
    "chat_history": [],
})

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'chat_history': '',
 'context': "Meanwhile, rookie safety Brian Branch, who has an ankle injury, is also set to miss his second consecutive game. The second-round pick out of Alabama was held out of practice for the entire week. With 25 tackles, a pick-six and 4 passes defended, he is Detroit's fourth-leading tackler.\n\nVeteran running back David Montgomery has carried the load in his absence, with back-to-back games of at least 100 yards rushing. Detroit has rushed for nine touchdowns through the first five games of the season, tying a team record.\n\nGibbs, who did not practice this week, also missed last week's 42-24 victory over the Carolina Panthers with the same injury. With 179 yards and a touchdown with an additional 70 receiving yards in his four appearances, he is Detroit's second-leading rusher.\n\nALLEN PARK, Mich. -- Detroit Lions running back Jahmyr Gibbs has been ruled out for Sunday's game against the Tampa Bay Buccaneers with a hamstring injury while Pro Bowl receive

In [48]:
conversational_qa_chain.invoke({
    "question": "Who is his running back coach?",
    "chat_history": [("What was Gibbs injury?", "Gibbs suffered a hamstring injury."),
                     ("Who are we talking about?"),("Gibbs is a running back for the Detroit Lions. He was their No. 12 overall selection in the 2021 NFL Draft and has been ruled out of their upcoming game against the Tampa Bay Buccaneers due to a hamstring injury. Despite high expectations, running back coach Scottie Montgomery is helping him develop at his own pace.")],
})

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'chat_history': '\nMe: What was Gibbs injury?\nAssistant: Gibbs suffered a hamstring injury.\nMe: W\nAssistant: h\nMe: G\nAssistant: i',
 'context': "Although expectations are high for Gibbs as the Lions' No. 12 overall selection, running back coach Scottie Montgomery is helping him develop at his own pace.\n\nGibbs, who did not practice this week, also missed last week's 42-24 victory over the Carolina Panthers with the same injury. With 179 yards and a touchdown with an additional 70 receiving yards in his four appearances, he is Detroit's second-leading rusher.\n\nALLEN PARK, Mich. -- Detroit Lions running back Jahmyr Gibbs has been ruled out for Sunday's game against the Tampa Bay Buccaneers with a hamstring injury while Pro Bowl receiver Amon-Ra St. Brown (abdomen) is set to make his return.\n\nVeteran running back David Montgomery has carried the load in his absence, with back-to-back games of at least 100 yards rushing. Detroit has rushed for nine touchdowns through the first f

### Add Memory